In [5]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import random
import pygame

# --- Constants ---
SCREEN_WIDTH = 1200
SCREEN_HEIGHT = 800
FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (220, 50, 50)
GREEN = (50, 220, 50)
BLUE = (50, 50, 220)
GRAY = (150, 150, 150)

In [138]:
import gymnasium as gym
from gymnasium.spaces import Box, Discrete,Tuple
import numpy as np
import pygame

# Define colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)  # Color for the trajectory

class CustomEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 60}

    def __init__(self):
        super().__init__()
        self.grid_size = 10
        self.obstacle_num = 8
        self.distance_threshold_goal = 0.3  # Tolerance for goal proximity

        # Define two separate thresholds for obstacle handling
        self.distance_threshold_penalty = 1  # Penalty zone threshold (larger value)
        self.distance_threshold_collision = 0.5  # Collision threshold (smaller value)
        self.penalty_factor = 25  # Penalty scaling factor

        self.steps = 0

        self.pre_distance = 0  # Previous distance to goal, used for reward shaping
        self.current_distance = 0  # Current distance to goal, used for reward shaping
        # Action space (dx, dy)
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)
        # Observation space (robot_x, robot_y, goal_x, goal_y)
        self.observation_shape = 2 + 2 + self.obstacle_num * 2 + 1 # Robot position, goal position, and obstacle positions
        self.observation_space = Box(low=0, high=np.array([self.grid_size for _ in range(self.observation_shape-1)]+[1]), shape=(self.observation_shape,), dtype=np.float32)

        
        # For rendering
        self.window = None
        self.clock = None
        self.cell_size = 50 # Pixels per grid unit
        self.trajectory_points = [] # New: List to store past robot positions

    def _get_obs(self):

        return np.concatenate(([self.robot_position]+ [self.goal_position]+self.obstacle_position+[np.array(self.steps)]),dtype=np.float32)

    def _get_info(self):
        return {
            "distance_to_goal": np.linalg.norm(self.robot_position - self.goal_position)
        }

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.robot_position = np.random.uniform(0, self.grid_size, size=2)
        self.goal_position = np.random.uniform(0, self.grid_size, size=2)
        self.obstacle_position = [np.random.uniform(0, self.grid_size, size=2) for _ in range(self.obstacle_num)]
        self.steps = 0
        self.trajectory_points = [self.robot_position.copy()] # New: Reset trajectory and add initial position
        self.pre_distance = np.linalg.norm(self.robot_position - self.goal_position)
        self.max_steps = 1000  # Set a maximum number of steps to prevent infinite loops
        return self._get_obs(), self._get_info()

    def _reward(self):
        terminated = False
        truncated = False
        if self.steps >= self.max_steps:
            reward -= 50
            truncated = True  # Truncate if max steps reached
        # Auxiliary Rewards -  distance to goal 
        self.current_distance = np.linalg.norm(self.robot_position - self.goal_position)
        reward = (self.pre_distance - self.current_distance)*30  # Reward shaping based on distance change
        self.pre_distance = self.current_distance

        # Obstacle handling with two thresholds
        for obs in self.obstacle_position:
            distance = np.linalg.norm(self.robot_position - obs)
            if distance < self.distance_threshold_collision:
                # Game over - collision detected
                reward -= 100  # Large negative reward for collision
                terminated = True
                break
            elif distance < self.distance_threshold_penalty:
                # Apply penalty for being too close to obstacle
                reward -= (self.distance_threshold_penalty - distance) * self.penalty_factor
        # 是否越界
        if np.any(self.robot_position == 0) or np.any(self.robot_position == self.grid_size):
            reward -= 100
            terminated = True  # Out of bounds
        if np.linalg.norm(self.robot_position - self.goal_position) < self.distance_threshold_goal:
            reward += 100
            terminated = True  # Reached the goal
        
        reward-=  0.5  # Small penaty for  each step to encourage efficiency

        return reward, terminated,truncated
        

    def step(self, action):
        stride = 0.3  # Base stride length
        self.robot_position += action * stride  # Scale the action to control speed
        self.robot_position = np.clip(self.robot_position, 0, self.grid_size)

        self.trajectory_points.append(self.robot_position.copy()) # New: Add current position to trajectory

        reward,terminated,truncated = self._reward()
        info = self._get_info()
        observation = self._get_obs()
        self.steps += 0.01
        return observation, reward, terminated, truncated, info

    def render(self):
        if self.window is None:
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (int(self.grid_size * self.cell_size), int(self.grid_size * self.cell_size))
            )
            pygame.display.set_caption("CustomEnv")
        if self.clock is None:
            self.clock = pygame.time.Clock()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                import sys
                sys.exit() # Exit the program
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1: # Left mouse button click
                    mouse_x, mouse_y = event.pos # Get mouse position in pixels
                    # Convert pixel coordinates to grid coordinates
                    new_goal_x = mouse_x / self.cell_size
                    new_goal_y = mouse_y / self.cell_size
                    self.goal_position = np.array([new_goal_x, new_goal_y], dtype=np.float32)
                    print(f"New goal set at grid position: {self.goal_position}")


        canvas = pygame.Surface((self.grid_size * self.cell_size, self.grid_size * self.cell_size))
        canvas.fill(WHITE)
        virus_image = pygame.image.load("病毒.png").convert_alpha()  # Load an image if needed, but not used here
        robot_image = pygame.transform.scale(virus_image, (int(self.cell_size * 0.8), int(self.cell_size * 0.8)))  # Scale the image
        # New: Draw the trajectory
        if len(self.trajectory_points) > 1:
            scaled_points = []
            for point in self.trajectory_points:
                scaled_points.append((int(point[0] * self.cell_size), int(point[1] * self.cell_size)))
            
            # Draw lines between consecutive points
            pygame.draw.lines(canvas, BLUE, False, scaled_points, 2) # Blue line, not closed, 2 pixels wide
            
            # Optionally, draw small circles at each point to emphasize
            for point_coord in scaled_points:
                pygame.draw.circle(canvas, BLUE, point_coord, 3) # Small blue circles

        # Draw robot
        pygame.draw.circle(
            canvas,
            RED,
            (int(self.robot_position[0] * self.cell_size), int(self.robot_position[1] * self.cell_size)),
            int(self.cell_size * 0.2)
        )
        # Draw obstacles
        for obs in self.obstacle_position:
            # pygame.draw.circle(
            #     canvas,
            #     GRAY,
            #     (int(obs[0] * self.cell_size), int(obs[1] * self.cell_size)),
            #     int(self.cell_size * 0.5)
            # )
            canvas.blit(robot_image, (int(obs[0] * self.cell_size), int(obs[1] * self.cell_size)))
        # Draw goal
        pygame.draw.circle(
            canvas,
            GREEN,
            (int(self.goal_position[0] * self.cell_size), int(self.goal_position[1] * self.cell_size)),
            int(self.cell_size * 0.3)
        )

        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.flip()
        self.clock.tick(self.metadata["render_fps"])

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()





In [137]:
import gymnasium as gym
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

env = CustomEnv()
policy_kwargs = dict(
    net_arch=[dict(pi=[64, 128,64], vf=[128, 256,128])],
    # activation_fn=torch.nn.ReLU  # 改为 ReLU，通常更适合稀疏奖励
)

model = PPO("MlpPolicy", env, verbose=1, ent_coef=0.01,policy_kwargs=policy_kwargs,tensorboard_log="./ppo_custom_env_tensorboard/")

eval_callback = EvalCallback(
    env,
    best_model_save_path='./logs/best_model/',
    log_path = './logs/',
    eval_freq=100000,  # 每1000步评估一次
    deterministic=True,
    render=False,
    n_eval_episodes=5,  # 每次评估5个episode
)

model.learn(total_timesteps=3000000)

# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(1000):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, dones, info = vec_env.step(action)
    # vec_env.render()


env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


g:\anaconda\envs\RL\lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 10 has 0 dimension(s)

In [225]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=2, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [128]:
env = CustomEnv()
obs,_ = env.reset()
print(obs)
for i in range(1000000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, teminated,_, info = env.step(action)
    print(action,info["distance_to_goal"])
    print(obs)
    print(action)
    print("Reward:", reward)
    print(env.robot_position)
    env.render()
    time.sleep(0.6)  # Control the frame rate
    if teminated:
        env.close()
        break
        print("Resetting environment")



ValueError: Box high must be a np.ndarray, integer, or float, actual type=<class 'list'>